# File formats

There are many different file formats in widespread use within data science. In this lecture, we will review common file formats and their trade-offs, and how to choose an appropriate file format. We will also review the mechanics of reading/parsing different file formats, and how to write to them.

- [CSV](https://docs.python.org/3/library/csv.html)
- [JSON](https://www.w3schools.com/js/js_json_intro.asp)
- [XML](https://www.w3schools.com/xml/xml_whatis.asp)
- [HDF5](https://support.hdfgroup.org/HDF5/Tutor/HDF5Intro.pdf)
- [SQLite3](https://docs.python.org/3/library/sqlite3.html)

These other formats may be touched on here but will be revisited when we look at big data and distributed computing.

- [Parquet](https://parquet.apache.org/documentation/latest/)
- [Avro](https://avro.apache.org/docs/current/)
- [Arrow](https://arrow.apache.org)

## Imports

### Standard library packages

In [ ]:
import os
import csv
import datetime
import decimal
import json
import sqlite3
import xml.etree.cElementTree as ET

### 3rd party packages

In [ ]:
! python3 -m pip install --quiet faker json2xml fastparquet fastavro rec_avro pyarrow

In [ ]:
import pendulum

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from faker import Faker

In [ ]:
from json2xml import json2xml
from json2xml.utils import readfromjson

In [ ]:
import fastavro 
from rec_avro import (to_rec_avro_destructive, 
                      from_rec_avro_destructive, 
                      rec_avro_schema)
import fastparquet

In [ ]:
import h5py
import tables

In [ ]:
from sqlalchemy import create_engine

## How to create fake data

### Create fake profiles using `Faker`

In [ ]:
fakes = [
    Faker('zh_CN'), 
    Faker('ar_SA'), 
    Faker('en_US'), 
]

In [ ]:
n = 3
p = [0.3, 0.2, 0.5]
np.random.seed(1)
locales = np.random.choice(len(fakes), size=n, p=p)

In [ ]:
profiles = [fakes[locale].profile() for locale in locales]

In [ ]:
profiles

### Make `pandas` data framee

In [ ]:
df = pd.DataFrame(profiles)

In [ ]:
df.iloc[0]

### Make comma delimited files

In [ ]:
df.to_csv('data/profiles.csv', index=False)

In [ ]:
! head -c 200 data/profiles.csv

### Make tab-delimited files

In [ ]:
df.to_csv('data/profiles.txt', index=False, sep='\t')

In [ ]:
! head -c 200 data/profiles.txt

### Make JSON files

In [ ]:
def converter(o):
    if isinstance(o, datetime.datetime):
        return o.__str__()
    if isinstance(o, decimal.Decimal):
        return o.__str__()

In [ ]:
with open('data/profiles.json', 'w') as f:
    json.dump(profiles , f, default=converter)

In [ ]:
! head -c 200 data/profiles.json

### Make XML files

In [ ]:
with open('data/profiles.xml', 'w') as f:
    data = readfromjson('data/profiles.json')
    f.write(json2xml.Json2xml({'employee': data}, wrapper="duke").to_xml())

In [ ]:
! head -c 200 data/profiles.xml

### Make AVRO files

In [ ]:
ps = json.load(open('data/profiles.json'))
avro_objects = [to_rec_avro_destructive(rec) for rec in ps]
with open('data/profiles.avro', 'wb') as f_out:
    fastavro.writer(f_out, fastavro.parse_schema(rec_avro_schema()), avro_objects)

In [ ]:
! head -c 200 data/profiles.avro

### Munge pandas data to be compratible with storage

In [ ]:
df.birthdate = pd.to_datetime(df.birthdate)
df = (
    df.current_location.
    apply(pd.Series).
    merge(df, left_index=True, right_index=True).
    drop('current_location', axis=1).
    rename({0: 'location_x', 1: 'location_y'}, axis=1)
)
df['location_x'] = df['location_x'].astype('float')
df['location_y'] = df['location_y'].astype('float')
df.website = df.website.apply(lambda s: ','.join(s))

### Make HDF5 files

In [ ]:
df.to_hdf('data/profiles.h5', key='duke')

In [ ]:
! head -c 200 data/profiles.h5

### Make Parquet files

In [ ]:
fastparquet.write('data/profiles.parq', df)

In [ ]:
! head -c 200 data/profiles.parq

### Make SQLite3 database files

In [ ]:
engine = create_engine('sqlite:///data/profiles.sqlite', echo=False)

In [ ]:
df.to_sql('duke', con=engine, if_exists='replace', index_label='id')

In [ ]:
! head -c 200 data/profiles.sqlite

## Reading data from different file formats

### CSV

#### When the CSV file can be read as is

In [ ]:
df = pd.read_csv('data/profiles.csv')

In [ ]:
df.head(1)

In [ ]:
df.loc[0]

#### When scrubbing of rows may be needed

In [ ]:
rows = []
with open('data/profiles.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        rows.append(row)

In [ ]:
list(map(len, rows))

In [ ]:
rows[:2]

In [ ]:
df = pd.DataFrame(rows[1:], columns=rows[0])

In [ ]:
df.head(1)

### Tab-delimited

Same as CSV, just change separator.

#### Direct reading into DataFrame

In [ ]:
df = pd.read_csv('data/profiles.txt', sep='\t')

In [ ]:
df.head()

#### Row by row processing

In [ ]:
rows = []
with open('data/profiles.txt') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        rows.append(row)

In [ ]:
list(map(len, rows))

### JSON

JSON is the most popular format for sharing information over the web. Most data retrieval APIs will return JSON.m

In [ ]:
with open('data/profiles.json') as f:
    profiles = json.load(f)

In [ ]:
len(profiles)

In [ ]:
profiles[0]

#### Using a REST API to retrieve JSON data

In [ ]:
if not os.path.exists('data/pokemon.json'):
    ! curl -o data/pokemon.json https://pokeapi.co/api/v2/pokemon/23

In [ ]:
with open('data/pokemon.json') as f:
    pokemon = json.load(f)

In [ ]:
pokemon.keys()

In [ ]:
pokemon['name']

In [ ]:
pokemon['abilities']

### Flatten nested JSON and extract fields to `pandas`

The [`json_normalize`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) function is useful for extracting nested fields from a JSON document. 

In [ ]:
import requests

In [ ]:
pd.json_normalize(pokemon['abilities'])

In [ ]:
pokemons = [requests.get(f'https://pokeapi.co/api/v2/pokemon/{i}').json() 
            for i in range(20,25)]

In [ ]:
pd.json_normalize(pokemons, ['moves', 'version_group_details'], ['name']).columns

In [ ]:
cols = ['name', 'move_learn_method.name',  'level_learned_at', 'version_group.name' ]
pd.json_normalize(pokemons, ['moves', 'version_group_details'], ['name'])[cols].drop_duplicates()

In [ ]:
df1 = pd.json_normalize(pokemons, 'abilities', ['name', 'order', 'weight', ['species', 'name']])

In [ ]:
df1

The `explode` and `apply` methods are useful if you have nested structures within a DataFrame.

In [ ]:
df_poke = pd.DataFrame(pokemons)

In [ ]:
df_poke.head(2)

Use `explode` to convert items in a list to separate rows, and `apply(pd.Series)` to convert items in a dictionary into separate columns.

In [ ]:
df1 = df_poke.abilities.explode().apply(pd.Series).reset_index(drop=True)
df1.head()

In [ ]:
df1.join(df1.ability.apply(pd.Series)).drop(columns = ['ability'])

### XML

In [ ]:
tree = ET.parse('data/profiles.xml')
root = tree.getroot()

In [ ]:
root.tag

In [ ]:
ET.dump(root)

In [ ]:
for employee in root:
    for elem in employee:
        print(f'{elem.tag:>20}: {elem.text}')
    break

In [ ]:
root.findall('.')

In [ ]:
root.findall('./')

In [ ]:
root.findall('.//')[:5]

In [ ]:
for item in root.findall('.//company'):
    print(item.text)

### HDF5

Like XML and JSON, HDF5 files store hierarchical data that can be annotated. The strong points of HDF5 are its ability to store large numerical data sets so that selective loading of parts of the data into memory for analysis is possible. HDF5 are also easy to use for people familiar with `numpy` and widely used in the scientific community.

There are two popular libraries for working with HDF5. Pandas uses `pytables`, and the stored schema can be quite unintuitive, but that does not matter since we usually just use Pandas to read it back in.

#### Pandas and `tables`

In [ ]:
f = tables.open_file('data/profiles.h5')

In [ ]:
f

In [ ]:
f.root.duke.axis0[:]

In [ ]:
f.root.duke.axis1[:]

In [ ]:
f.root.duke.block0_items[:]

In [ ]:
f.root.duke.block0_values[:]

In [ ]:
f.close()

#### Reading into `pandas`

In [ ]:
df = pd.read_hdf('data/profiles.h5')

In [ ]:
df

#### Using `h5py`

For actually working directly with HDF5, I find `h5py` more intuitive.

In [ ]:
filename = 'data/si.h5'
if os.path.exists(filename):
    os.remove(filename)
f = h5py.File(filename, 'w')

In [ ]:
start = pendulum.datetime(2019, 8, 31)
stop = start.add(days=3)
for day in pendulum.period(start, stop):
    g = f.create_group(day.format('ddd'))
    g.attrs['date'] = day.format('LLL')
    g.attrs['analyst'] = 'Mario'
    for expt in range(3):
        data = np.random.poisson(size=(100, 100))
        ds = g.create_dataset(f'expt-{expt:02d}', data=data)

In [ ]:
f = h5py.File(filename, 'r')

In [ ]:
list(f.keys())

In [ ]:
list(f['Sat'].attrs.keys())

In [ ]:
f['Sat'].attrs['analyst']

In [ ]:
f['Sat'].attrs['date']

In [ ]:
list(f['Sat'].keys())

In [ ]:
f['Sat']['expt-01'][5:10, 5:10]

In [ ]:
f['Sat']['expt-01'][5:10, 5:10].sum(axis=0)

In [ ]:
f.close()

## Avro

In [ ]:
! python3 -m pip install --quiet fastavro rec_avro

In [ ]:
%%bash --out s
fastavro --schema data/profiles.avro

In [ ]:
schema = eval(s.replace('true', 'True'))

In [ ]:
schema

In [ ]:
with open('data/profiles.avro', 'rb') as f:
    avro_reader = fastavro.reader(f, reader_schema=schema)
    for record in avro_reader:
        print(record)

#### Avro to JSON

In [ ]:
with open('data/profiles.avro', 'rb') as f:
    avro_reader = fastavro.reader(f, reader_schema=schema)
    for record in avro_reader:
        print(from_rec_avro_destructive(record))

### Parquet

In [ ]:
! python3 -m pip install --quiet fastparquet

In [ ]:
parq = fastparquet.ParquetFile('data/profiles.parq')

In [ ]:
parq.columns

In [ ]:
df = parq.to_pandas()

In [ ]:
df.head(1)

#### Reading directly in `pandas`

In [ ]:
df = pd.read_parquet('data/profiles.parq')

In [ ]:
df.head(1)

## SQL

A relatinal databse isn't really a filetype, but SQLite3 stores data as a simple file.

In [ ]:
conn = sqlite3.connect('data/profiles.sqlite')
c = conn.cursor()

In [ ]:
c.execute("SELECT * FROM sqlite_master WHERE type='table'")
c.fetchall()

In [ ]:
c.execute('SELECT * FROM duke')
c.fetchone()

In [ ]:
conn.close()